In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-assistants-with-gemma/submission_categories.txt
/kaggle/input/data-assistants-with-gemma/submission_instructions.txt


# Using Gemma to Answer Common Python Questions

## Overview
In this notebook, we demonstrate how to use Google's Gemma model to effectively answer common questions about the Python programming language. This task is crucial for supporting Python learners and developers in understanding key concepts and techniques.

## Model Introduction
The Gemma model, developed by Google, is a state-of-the-art large language model that excels in natural language understanding and generation. We leverage this model to provide accurate and insightful answers to frequently asked Python-related questions.


## Install Required Libraries

In [2]:
!pip install -U transformers

In [3]:
!pip install ipywidgets

## Import Modules

In [4]:
# Import necessary modules
from transformers import AutoTokenizer, AutoModelForCausalLM
from kaggle_secrets import UserSecretsClient
from ipywidgets import widgets, Output
from IPython.display import display, clear_output

## Authentication and Loading the Gemma Model

To access Gemma, we need to authenticate using the Hugging Face token available through Kaggle secrets. This ensures secure access to the model's API.

In [5]:
# Retrieve Hugging Face token securely
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("huggingface_token")

# Load the tokenizer and model with the token for authentication
model_name = "google/gemma-2b-it"  
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token)


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

# Few-Shot Priming of the Model

To help Gemma understand the context of Python programming better, we utilize a technique called "few-shot learning." We provide a set of example questions and answers that serve as a guide for the model. This primes the model to generate responses that are more in line with the Python programming domain.

In [6]:
# Define few-shot examples
few_shot_prompts = [
    "What is a variable in Python? A variable in Python is a name that refers to a value.",
    "What is a list in Python? A list is a collection of items that can be changed and ordered. It is created using square brackets.",
    "How do I write a for loop in Python? A for loop in Python allows you to iterate over a sequence. It is written using 'for' followed by a variable name and the sequence.",
    "What is a function in Python? A function is a block of code that performs a task when called. It is defined using the def keyword.",
    "How do I open a file in Python? You can open a file in Python using the open() function.",
   
]

# Function to prime the model with few-shot examples
def prime_model_with_few_shot_examples(model, tokenizer, few_shot_prompts):
    combined_prompts = "\n".join(few_shot_prompts)
    inputs = tokenizer.encode(combined_prompts, return_tensors='pt', padding=True)
    # Generate output from the model to prime it with the few-shot context
    _ = model.generate(inputs, max_length=512, num_return_sequences=1)
    print("Model has been primed with few-shot examples.")

# Execute the priming function
prime_model_with_few_shot_examples(model, tokenizer, few_shot_prompts)

Model has been primed with few-shot examples.


## Interactive Widgets for User Questions

We provide an interactive experience for users to input their own Python questions and receive answers generated by the Gemma model.


In [7]:
# Create an output area for displaying the answers
output_area = Output()

# Function to fetch and display answers from Gemma
@output_area.capture(clear_output=True, wait=True)
def fetch_answer(b):
    try:
        question = question_input.value
        inputs = tokenizer.encode(question, return_tensors='pt')
        outputs = model.generate(inputs, max_length=200, num_return_sequences=1)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Answer: {answer}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Set up widgets for user interaction
question_input = widgets.Text(
    value='What is a function in Python?',
    placeholder='Type your Python question here...',
    description='Question:',
    disabled=False
)
button = widgets.Button(description="Get Answer")
button.on_click(fetch_answer)

# Display the widgets
display(question_input, button, output_area)


Text(value='What is a function in Python?', description='Question:', placeholder='Type your Python question he…

Button(description='Get Answer', style=ButtonStyle())

Output()

## Evaluation of Model Performance

To gauge the effectiveness of the Gemma model in answering Python questions, we assess the generated answers based on several criteria:

- **Accuracy**: Do the responses correctly address the questions posed?
- **Relevance**: Are the answers on-topic and useful for the question context?
- **Clarity**: Is the information presented in a clear and understandable manner?

A qualitative evaluation has been conducted to ensure that the answers meet these criteria. We've tested the model with a set of diverse questions ranging from basic to advanced Python concepts to evaluate its robustness and versatility.

### Feedback

Based on initial observations:

- The Gemma model performs well with questions about Python syntax and operations.


## Conclusion and Future Directions

This notebook has showcased the potential of Google's Gemma model as a supportive tool for those learning Python. By providing a means to answer common Python questions interactively, we've opened a pathway for learners to engage with AI in their educational journey.

### Reflections

Through this project, we've observed Gemma's capabilities and identified areas for growth. The model excels at basic Q&A tasks but has room for improvement in nuanced language understanding and complex problem-solving.

### Future Work

In moving forward, the following areas will be explored:

- Incorporating a broader range of programming topics and more complex questions.
- Experimenting with model fine-tuning to tailor responses to the Python programming domain specifically.


## Contributions

This project is the work of [Najmul Hasan](https://www.kaggle.com/najmulhasancsmath), crafted to enhance Python learning experiences on Kaggle. The endeavor demonstrates practical applications of the Gemma model, with the aim of contributing valuable insights to the broader machine learning and educational community.
